In [0]:
!pip install openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import re
from datetime import datetime
import warnings
from collections import OrderedDict
import os
import yaml
warnings.filterwarnings('ignore')

In [0]:
%run "./fetch_data"

In [0]:
%run "./get_hl_from_skuname"

In [0]:
with open('./config.yml', 'r', encoding='utf-8') as file:
    params_dict = yaml.safe_load(file)
ws_params_dict_inv = params_dict['ws_params_dict_inv']  # inv数据使用列名
ws_params_dict = params_dict['ws_params_dict']     # stw数据 列名
ws_params_dict_str = params_dict['ws_params_dict_str']   # str列名

In [0]:
def get_sku_hl_mapping_table():
    sku_hl_mapping = spark.sql(
    """
    WITH CIO AS (
    SELECT
        DISTINCT cio_id AS cio_code,
        c_product as cio_name,
        regexp_replace(capacity, '([a-zA-Z])', '') / 1000.0 as container_L,
        -- trim(C2.Seg) AS Segment,
        packagesize
    FROM
        bees_products_ods.product_center_cio_sku C1
        LEFT JOIN bees_products_ods.product_center_cio_mapping C2 ON C2.cio_code = C1.cio_id
    WHERE
        deleted = 0
        AND regexp_replace(capacity, '([a-zA-Z])', '') > 0
        AND C2.Seg IS NOT NULL
        AND COALESCE(packagesize, 0) > 0
    )
    SELECT
    DISTINCT SC.sap_sku_code as SKUCode, K1.cio_code
    ,K1.cio_name
    ,K1.container_L * K1.packagesize/100 HL
    FROM
    CIO K1
    JOIN bees_products_ods.product_center_ref_sap_to_cio SC ON SC.cio_sku_code = K1.cio_code
    """
    ).toPandas()
    return sku_hl_mapping

def get_sku_sap_wccs_code_mapping_table():
    wccs_productid_sap_sku_mapping = spark.sql(
    """
    select distinct product_id, cast(sap_skudm as int) as SKUCode from wccs_ods.bas_material
    """).toPandas()
    return wccs_productid_sap_sku_mapping     

In [0]:
used_wccs_sku_list, sku_filtered, product_id_mapping = get_used_brand_family() # sku_filtered是过滤了四个brand的sku mapping表；product_id_mapping是全量的sku hl mapping表
begin_inv = get_inv_data(used_wccs_sku_list, sku_filtered, used_qty_col='期初库存')
stw = get_stw_data(start='202101', end='202410')
ws_str = get_str_data(product_id_mapping,start='202101', end='202410')

没对应上SKUCode/HL的wccs product id ['P02816' 'P02747' 'P02746' 'P02931' 'P03012' 'P03002' 'P02896' 'P02820'
 'P02650' 'P02843' 'P02708' 'P03001' 'P03071' 'P03020' 'P00012' 'P00007'
 'P01040' 'P00024' 'P00043' 'P00033' 'P00038' 'P00040' 'P00032' 'P00028'
 'P00020' 'P00025' 'P00015' 'P00034' 'P00036' 'P00031' 'P00029' 'P00019'
 'P00014' 'P00027' 'P00035' 'P00830' 'P00211' 'P00022' 'P00030' 'P00627'
 'P02935' 'P03006' 'P02757' 'P00315' 'P00047' 'P00045' 'P00076' 'P00046'
 'P00055' 'P00098' 'P00053' 'P00051' 'P00050' 'P00064' 'P00062' 'P00058'
 'P00059' 'P01018' 'P00061' 'P00063' 'P00867' 'P00078' 'P02791' 'P02799'
 'P00073' 'P00074' 'P00056' 'P00065' 'P00067' 'P00072' 'P00057' 'P00233'
 'P00408' 'P00329' 'P00307' 'P00267' 'P00314' 'P00272' 'P00317' 'P00359'
 'P00444' 'P00481' 'P00343' 'P00704' 'P00249' 'P00301' 'P00254' 'P00260'
 'P00327' 'P00273' 'P00278' 'P00446' 'P00299' 'P00316' 'P00225' 'P00633'
 'P00401' 'P00631' 'P00469' 'P00295' 'P00653' 'P00558' 'P00231' 'P00854'
 'P00585' 'P00308' '

In [0]:
ws_attributes2 = stw[['payercode', 'payername', 'payer_bu_name', 'payer_region_name']].drop_duplicates(subset=['payercode'])
ws_attributes = ws_str[['payercode', 'payer_bu_name', 'payer_region_name']].drop_duplicates(subset=['payercode'])
ws_attributes['payercode'] = ws_attributes['payercode'].astype(int)
ws_str['payercode'] = ws_str['payercode'].astype(int)

ws_attributes = ws_attributes.set_index('payercode')
ws_attributes2 = ws_attributes2.set_index('payercode')

ws_bu_dict = dict(ws_attributes['payer_bu_name'])
ws_region_dict = dict(ws_attributes['payer_region_name'])
# ws_name_dict = dict(ws_attributes2['payername'])
ws_code_name_payercode_df = get_ws_name_payercode_mapping()
ws_name_dict = dict(ws_code_name_payercode_df['payername'])

In [0]:
def make_monthly_horizontal_df(lst, name):
    df = pd.DataFrame(lst, columns=[name]).T
    df.columns = [x + 1 for x in df.columns]
    df = df.add_prefix('month_')
    return df


def get_initial_inv(data, **params):
    ws_payer_code = params['ws_payer_code']
    ws_id_name = params['ws_id_name']
    ws_inv = data.query(f"{ws_id_name} == '{ws_payer_code}'")

    if params['agg_col'] == 'hl':
        ws_inv[params['sku_name']] = ws_inv[params['sku_name']].str.upper()
        ws_inv[params['sku_name']] = ws_inv[params['sku_name']].apply(lambda x: re.sub(r'\s|[\d]%', '', x))
        sku_name_list = list(ws_inv[params['sku_name']].unique())
        sku_hl_mapping_table = get_sku_hl_mapping_table(sku_name_list)
        ws_inv = pd.merge(ws_inv, sku_hl_mapping_table[['sku_name', 'hl']], left_on=params['sku_name'], right_on='sku_name')
    else:
        sku_hl_mapping_table = pd.DataFrame()                                                                          
    ws_inv_agg = ws_inv.groupby([ws_id_name])[params['agg_col']].sum()
    ws_inv_agg = ws_inv_agg.astype(float)

    return ws_inv_agg, sku_hl_mapping_table


def get_num_list(stw, **params):
    # STW/STR 12个月的值
    ws_stw = stw.query(f"{params['ws_id_name']} == {params['ws_payer_code']}")
    if len(ws_stw) > 0:
        if params['agg_col'] == 'hl':   # ws stw表里的agg_col是stw_hl
            ws_stw[params['sku_name']] = ws_stw[params['sku_name']].str.upper()
            ws_stw[params['sku_name']] = ws_stw[params['sku_name']].apply(lambda x: re.sub(r'\s|[\d]%', '', x))
            sku_name_list = list(ws_stw[params['sku_name']].unique())
            sku_hl_mapping_table = get_sku_hl_mapping_table(sku_name_list)
            ws_stw = pd.merge(ws_stw, sku_hl_mapping_table[['sku_name', 'hl']], left_on=params['sku_name'], right_on='sku_name')
        else:
            sku_hl_mapping_table = pd.DataFrame()                                                                        
        ws_stw_agg = ws_stw.groupby([params['month_col']])[params['agg_col']].sum()
        ws_stw_agg = ws_stw_agg.astype(float)       
        ws_stw_agg = ws_stw_agg.reindex(stw_yr_month_index).fillna(0)
        ws_stw_lst = ws_stw_agg.to_list()
    else:
        ws_stw_lst = [0] * len(stw_yr_month_index)
        sku_hl_mapping_table = pd.DataFrame()
    return ws_stw_lst, sku_hl_mapping_table

In [0]:
def get_stw_monthly_balance(begin_inv, stw: list, out_str: list, inv_expire_month=6, simu_months=12):
    """
    begin_inv:202212期末库存=2023年1月期初库存
    stw: 2023年12个月的stw进货量
    out_str:2023年12个月的出货量
    simu_months: 模拟的月份数，默认是1年即12个月
    result: 期初库存每个月的结余
    """
#     # 测试数据
#     begin_inv = 80
#     stw = [80]*2 + [100]*10     # 进货
#     out_str = [80]*7 + [80]*5

    begin_inv_remain = begin_inv
    stw_remain = stw.copy()             #  截止到当前月每个月的进货量还剩多少                
    stw_remain_monthly = []             # 每个月的结果合集
    begin_inv_remain_monthly = []       # 每个月的结果合集
    #     begin_inv_remain_lst = [None] * 12
    #     stw_remain_list = []        # 每个月进货stw的剩余
    str_remain_list = []
    for i in range(simu_months):
        # if i >= 34:
        #     print(f"month {i}")
        stw_remain_list = []
        if (out_str[i] <= begin_inv_remain) and (i <= (inv_expire_month - 1)) and (out_str[i] > 0):     # 期初库存能扣到6月
            begin_inv_remain -= out_str[i]  # 期初（年初）库存的剩余
            stw_remain_list = stw_remain.copy()     # 每个月stw的剩余
            stw_remain_list = [stw_remain_list[stw_month] if stw_month <= i else None for stw_month in range(simu_months) ]
            str_remain = 0
            str_remain_list.append(str_remain)
        else:
            if (begin_inv_remain > 0) and (i <= (inv_expire_month - 1)) and (out_str[i] > 0):
                str_remain = out_str[i] - begin_inv_remain     # 这个月还欠多少货
                begin_inv_remain = 0
            else:
                str_remain = out_str[i]
            for j in range(len(stw_remain)):
                if i > (inv_expire_month + j):
                    pass                             # 如果到了8月，1月进货的stw已经不能用了
                else:
                    if stw_remain[j] > 0:
                        delta = min(str_remain, stw_remain[j])   # 这个月的剩余需求和当前月的进货余量
                    else:
                        delta = 0         
                    str_remain -= delta
                    stw_remain[j] -= delta
                if j <= i:
                    stw_remain_list.append(stw_remain[j])    #  大于当前月的进货不能用
                else:
                    stw_remain_list.append(None) 
                if j == i:
                    str_remain_list.append(str_remain)       # 用于检验这个月str需求是否全被满足了
        stw_remain_monthly.append(stw_remain_list)
        begin_inv_remain_monthly.append(begin_inv_remain)
    #     return stw_remain_monthly, begin_inv_remain_monthly, str_remain_list
    stw_replenish_df = make_monthly_horizontal_df(stw, 'STW(+)')
    str_remain_df = make_monthly_horizontal_df(str_remain_list, 'str_remain')
    out_str_df = make_monthly_horizontal_df(out_str, 'STR(-)')
    begin_inv_remain_df = make_monthly_horizontal_df(begin_inv_remain_monthly, 'openning_balance')
    stw_balance_df = pd.DataFrame(stw_remain_monthly, index=range(1, simu_months + 1), columns=range(1, simu_months + 1)).T
    stw_balance_df = stw_balance_df.add_prefix('month_')
    stw_balance_df.index = stw_balance_df.index.map(lambda x: str(x) + '_STW_balance')
    output = pd.concat([stw_replenish_df, out_str_df, begin_inv_remain_df, stw_balance_df, str_remain_df])
    year_begin_inv_balance = pd.DataFrame([begin_inv] + [None]*(len(output)-1), index=output.index, columns=['Openning Balance'])
    return pd.concat([year_begin_inv_balance, output], axis=1)

def get_result_df(begin_inv, stw, ws_str, expire_month, simu_months, ws_params_dict_inv, ws_params_dict, ws_params_dict_str):
    ws_begin_inv,sku_hl_mapping_inv = get_initial_inv(begin_inv, **ws_params_dict_inv)   # 获取年期初库存
    ws_stw_lst, _ = get_num_list(stw, **ws_params_dict)                  # 获取每个月的进货量stw
    ws_str_lst,sku_hl_mapping_str = get_num_list(ws_str, **ws_params_dict_str)           # 获取每个月的出货量str
#         if min(len(ws_stw_lst), len(ws_str_lst)) < simu_months:
#             continue
    ws_begin_inv = 0 if len(ws_begin_inv) == 0 else ws_begin_inv.values[0]
    result_df = get_stw_monthly_balance(begin_inv=ws_begin_inv, stw=ws_stw_lst, out_str=ws_str_lst, inv_expire_month=expire_month, simu_months=simu_months)
    return result_df,sku_hl_mapping_inv,sku_hl_mapping_str

In [0]:
def generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='stw_qty', inv_expire_month=6, simu_months=12):
    """
        used_ws_dict: 要模拟库存的经销商
        begin_inv:年初库存
        stw: 所有经销商的stw数据
        ws_str: 所有经销商的str出货数据
        col: stw_qty箱数 or hl百升数; 算库存箱数还是百升数
        inv_expire_month: 多少个月后酒库存的就过期了,如果是int类型则所有sku默认一样的，如果是个dict，则是by brand有效期不同
        simu_months: 模拟的月份数，默认是1年即12个月
    """
    # dt_string = datetime.now().date().strftime("%m%d")
    # if isinstance(inv_expire_month, int):
    #     file = pd.ExcelWriter(f'result_df/result_data_{col}_{dt_string}_{inv_expire_month}.xlsx')
    # else:
    #     file = pd.ExcelWriter(f'result_df/result_data_{col}_{dt_string}_by_brand_expired.xlsx')

    ws_brand_df_list = []                                # Final明细结果
    if col == 'hl':                                      # 通过洗sku的名字算hl
        ws_params_dict_inv['agg_col'] = 'hl'
        ws_params_dict['agg_col'] = 'stw_hl'
        ws_params_dict_str['agg_col'] = 'hl'
    elif col == 'hl_src':                                # 取数算的百升
        ws_params_dict_inv['agg_col'] = 'inv_hl'
        ws_params_dict['agg_col'] = 'stw_hl'
        ws_params_dict_str['agg_col'] = 'str_hl'
    else:                                                # qty箱数
        pass    
        
    sku_hl_mapping_df = pd.DataFrame()
    result_df_dict = {}
    for payer_code, ws_code in used_ws_dict.items():
        print(f"ws_code:payer_code {ws_code}:{payer_code}")
        #     ws_stw_lst = ws_stw_agg.to_list()
        ws_params_dict_inv['ws_payer_code'] = ws_code
        ws_params_dict['ws_payer_code'] = payer_code
        ws_params_dict_str['ws_payer_code'] = payer_code
        if isinstance(inv_expire_month, int):    # 所有SKU/brand默认有相同的保鲜期
            result_df = get_result_df(begin_inv, stw, ws_str, inv_expire_month, simu_months)
        else:    # 一个brand 一个结果，最后加起来汇总成一个经销商的
            result_df = pd.DataFrame()   # result_df: 一个ws把所有brand的库存加起来的表
            for brand, brand_expire_month in inv_expire_month.items():
                begin_inv_brand = begin_inv[begin_inv[ws_params_dict_inv['brand_col']] == brand]
                stw_brand = stw[stw[ws_params_dict['brand_col']] == brand]
                ws_str_brand = ws_str[ws_str[ws_params_dict_str['brand_col']] == brand]
                result_df_brand,sku_hl_mapping_inv,sku_hl_mapping_str = get_result_df(begin_inv_brand, stw_brand, ws_str_brand, brand_expire_month, simu_months, ws_params_dict_inv, ws_params_dict, ws_params_dict_str)
                # result_df: 一个ws把所有brand的库存加起来的表
                result_df = result_df.combine(result_df_brand, lambda s1, s2: s1.fillna(0) + s2.fillna(0))
                result_df_brand.insert(0, 'expire_month', brand_expire_month)
                result_df_brand.insert(0, 'brand', brand)
                result_df_brand.insert(0, 'region', ws_region_dict[payer_code])
                result_df_brand.insert(0, 'bu', ws_bu_dict[payer_code])
                result_df_brand.insert(0, 'ws_code', payer_code)   # 也可以改payer_code
                result_df_brand.insert(0, 'ws_name', ws_name_dict.get(payer_code))
                ws_brand_df_list.append(result_df_brand)
        # result_df.to_excel(file, sheet_name=ws_code)
        result_df.insert(0, 'expire_month', None)
        result_df.insert(0, 'brand', "all")
        result_df.insert(0, 'region', ws_region_dict[payer_code])
        result_df.insert(0, 'bu', ws_bu_dict[payer_code])
        result_df.insert(0, 'ws_code', payer_code)   # 也可以改payer_code/ws_code
        result_df.insert(0, 'ws_name', ws_name_dict.get(payer_code))
        ws_brand_df_list.append(result_df)
        result_df_dict[ws_code] = result_df
        if len(sku_hl_mapping_inv) > 0:
            sku_hl_mapping_inv['type'] = 'inv'
            sku_hl_mapping_inv['ws_code'] = ws_code
            sku_hl_mapping_df = pd.concat([sku_hl_mapping_df, sku_hl_mapping_inv])
        if len(sku_hl_mapping_str) > 0:
            sku_hl_mapping_str['type'] = 'str'
            sku_hl_mapping_str['ws_code'] = ws_code
            sku_hl_mapping_df = pd.concat([sku_hl_mapping_df, sku_hl_mapping_str])
    # if len(sku_hl_mapping_df) > 0:
    #     sku_hl_mapping_df.to_excel(file, sheet_name='sku_hl_mapping')
    return ws_brand_df_list, result_df_dict

In [0]:
month_cnt = len(stw['billingyearmonth'].unique())      # 2021.1到24年10月是46个月；每个经销商都要46个月
print(f"stw月份数：{month_cnt}")
print(f"str月份数：{len(ws_str['month'].unique())}")
stw_yr_month_index = stw['billingyearmonth'].drop_duplicates().sort_values()
# str_yr_month_index = ws_str['month'].drop_duplicates().sort_values()   # 长得是一样的

stw月份数：46
str月份数：46


In [0]:
# ws_code_mapping_table = spark.sql("select distinct wholesalerid as ws_code, int(Pay_to) as payercode  from bcc_baseline_ods.wccs_ws").toPandas()
ws_code_mapping_table = pd.read_excel('/dbfs/FileStore/tables/srf/fifo_inv_simu/src_data/ws_code_payer_code_mapping.xlsx').drop_duplicates(subset=['payercode']) # 进销存excel里的
# ws_code_mapping_table.groupby('payercode').agg({'ws_code':pd.Series.nunique}).sort_values(by='ws_code', ascending=False)  # 检查一对一关系
ws_list = ws_str['payercode'].drop_duplicates()
used_ws_code_mapping_df = pd.merge(ws_list, ws_code_mapping_table, on='payercode', how='left').set_index('payercode')    # w码有缺失值

In [0]:
# used_ws_dict = {30018687:'W00013686', 30012653: 'W00008121'}
used_ws_dict = dict(used_ws_code_mapping_df['ws_code'])
inv_unit = 'hl_src'         # stw_qty 或其他值，都是指默认值，箱数/////hl_src是原始数据带的百升数/////hl是根据sku洗出来的百升数
# generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='stw_qty',inv_expire_month=inv_expire_month, simu_months=month_cnt)
inv_expire_month = 9    # 库存9个月后失效，之前是6个月
brand_expire_month_dict = {'BUD': inv_expire_month, 'HBI': inv_expire_month, 'HBO': inv_expire_month, 'HKOW':inv_expire_month, 'ISP': inv_expire_month}
# 默认月份一样
# result_df_dict = generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='hl_src',inv_expire_month=inv_expire_month, simu_months=month_cnt)
# 每个brand的保质期都不一样
ws_brand_df_list, result_df_dict = generate_fifo_inventory_balance(used_ws_dict, begin_inv, stw, ws_str, col='hl_src',inv_expire_month=brand_expire_month_dict, simu_months=month_cnt)

ws_code:payer_code W00014804:30019705
ws_code:payer_code W00015499:30020351
ws_code:payer_code W00014088:30019043
ws_code:payer_code W00004564:30003672
ws_code:payer_code W00012457:30017071
ws_code:payer_code W00014033:30018978
ws_code:payer_code W00006907:30012857
ws_code:payer_code W00001455:30002866
ws_code:payer_code W00015431:30020286
ws_code:payer_code W00014015:30018934
ws_code:payer_code W00013796:30018795
ws_code:payer_code W00001794:30003266
ws_code:payer_code W00006891:30011189
ws_code:payer_code W00011626:30016081
ws_code:payer_code W00012325:30017122
ws_code:payer_code W00001461:30002767
ws_code:payer_code W00011188:30015701
ws_code:payer_code W00015380:30020230
ws_code:payer_code W00012920:30017846
ws_code:payer_code W00004379:30002248
ws_code:payer_code W00011553:30015993
ws_code:payer_code W00004882:30003317
ws_code:payer_code W00000468:30008389
ws_code:payer_code W00001945:30003227
ws_code:payer_code W00007981:30011164
ws_code:payer_code W00003877:30003086
ws_code:paye

In [0]:
ws_brand_df = pd.concat(ws_brand_df_list).reset_index()
ws_brand_df_spark = spark.createDataFrame(ws_brand_df)
ws_brand_df_spark.write.mode("overwrite").option('overwriteSchema',True).parquet(f'/mnt/srf/fifo_inventory_details_month{inv_expire_month}')

In [0]:
ws_brand_df_spark = ws_brand_df_spark.withColumnRenamed('Openning Balance', 'Openning_Balance')
ws_brand_df_spark.write.mode("overwrite").format("delta").option('overwriteSchema',True).saveAsTable(f"finance_ds_inventory_dmt.fifo_inventory_details_{inv_expire_month}")